# Support Vector Machine
Support vector machines are supervised machine learning methods that can be used for classification and regression. 

The SVC classifier and SVR regressor can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames/Series as the input.

For information on converting your dataset to cuDF documentation: https://docs.rapids.ai/api/cudf/stable/

For more information about cuML's Support Vector Classifier: https://docs.rapids.ai/api/cuml/stable/

## 1. Preparation

In [ ]:
import numpy as np
import cuml.svm 
import sklearn.svm

from sklearn.datasets import make_gaussian_quantiles
from sklearn.datasets import make_friedman1
from sklearn.model_selection import train_test_split
from cuml.metrics.regression import r2_score

## 2. Support vector classification
Currently cuML supports binary classification (C-Support Vector Classification).

### Generate data

In [ ]:
n_samples = 20000
n_features = 200

X, y = make_gaussian_quantiles(n_samples=n_samples, n_features=n_features, n_classes=2)

X_train, X_test, y_train, y_test = train_test_split(X, y)

### Define parameters

In [ ]:
C = 1
tol = 1e-3
kernel = 'rbf'
gamma = 'scale'

### cuML Model

In [ ]:
%%time
cumlSVC = cuml.svm.SVC(kernel=kernel, C=C, tol=tol, gamma=gamma)
cumlSVC.fit(X_train, y_train)

### Scikit-learn Model

In [ ]:
%%time
sklSVC = sklearn.svm.SVC(kernel=kernel, C=C, tol=tol, gamma=gamma)
sklSVC.fit(X_train, y_train)

### Prediction

In [ ]:
%%time
cuml_pred = cumlSVC.predict(X_test)

In [ ]:
%%time
skl_pred = sklSVC.predict(X_test)

### Compare Accuracy

In [ ]:
cuml_accuracy = np.sum(cuml_pred.to_array()==y_test) / y_test.shape[0] * 100
skl_accuracy = np.sum(skl_pred==y_test) / y_test.shape[0] * 100
print("Accuracy: cumlSVC {}%, sklSVC {}%".format(cuml_accuracy, skl_accuracy))

### Notes
- The time measurements will be inaccurate for the first run. You can re-run the cells to get a better estimate of the execution time.

- Currently the output of the prediction is a cuDF Series object. You can use the `to_array()` method to create a numpy array.

- The training algorithm uses a cache in GPU memory to accelerate training. You can specify the size (in MiB) using the cache_size argument. This is more relevant for training with larger input size.

- Similar to other cuML algorithms, cuML SVC is optimized both for single and double precision input data. If your problem allows it, then using single precision input can improve the execution time

In [ ]:
%%time
cumlSVC = cuml.svm.SVC(kernel=kernel, C=C, tol=tol, gamma='scale', cache_size=2000)
cumlSVC.fit(X_train.astype(np.float32), y_train)

## 3. Support vector regression

cuML supports epsilon Support Vector Regression, where the epsilon parameter defines the radius of the epsilon-tube around the target values. If a prediction falls within this tube, then no penalty is associated.

### Generate data

In [ ]:
n_samples = 20000
n_features = 20

X, y = make_friedman1(n_samples=n_samples, n_features=n_features, noise=0.0, random_state=137)

X_train, X_test, y_train, y_test = train_test_split(X, y)

### Model parameters

In [ ]:
C = 1
tol = 1e-3
kernel = 'rbf'
gamma =  'scale'
epsilon = 0.1

### cuML model

In [ ]:
%%time
cumlSVR = cuml.svm.SVR(kernel=kernel, C=C, tol=tol, gamma=gamma, epsilon=epsilon)
cumlSVR.fit(X_train, y_train)

### Scikit-learn model

In [ ]:
%%time
sklSVR = sklearn.svm.SVR(kernel=kernel, C=C, tol=tol, gamma=gamma, epsilon=epsilon)
sklSVR.fit(X_train, y_train)

### Prediction

In [ ]:
%%time
cuml_pred = cumlSVR.predict(X_test)

In [ ]:
%%time
skl_pred = sklSVR.predict(X_test)

### Compare accuracy

In [ ]:
cuml_score = r2_score(y_test, cuml_pred) 
skl_score = r2_score(y_test, skl_pred)
print("R2 score: cumlSVR {}, sklSVR {}".format(cuml_score, skl_score))

### Notes
The same notes apply as for SVC: you can improve the training time if you increase the cache size, or change to lower precision.


In [ ]:
%%time
cumlSVR = cuml.svm.SVR(kernel=kernel, C=C, tol=tol, gamma='scale', cache_size=1024)
cumlSVR.fit(X_train.astype(np.float32), y_train)

In [ ]:
cumlSVR.score(X_test.astype(np.float32), y_test.astype(np.float32))